In [1]:
import struct
import sys
import os
import shutil

In [2]:
def parse_bin_file(csi_raw_video: str, width: int = 320, height: int = 180, fps: int = 30):
    # Open the file
    f = open(csi_raw_video, 'rb')

    # If the file is not opened successfully, then output the error message and return
    if not f:
        raise Exception("couldn't open file {}".format(csi_raw_video))

    # Move the file pointer back to the beginning of the file to start reading the file
    f.seek(0, 0)

    # The bin file is roughly composed of four parts, the first is the total length of 
    # CV data and the total length of CSI data composed of two size_t.
    cv_total_size = struct.unpack('<Q', f.read(8))[0]
    csi_total_size = struct.unpack('<Q', f.read(8))[0]

    # The second part is the CV data, which is the raw video data
    cv_data = f.read(cv_total_size)

    # The third part is the CSI data, which is the raw CSI data
    csi_data = f.read(csi_total_size)

    # Close the file first
    f.close()

    # Return the data
    return {
        "cv_data": cv_data,
        "cv_data_len": cv_total_size,
        "csi_data": csi_data,
        "csi_data_len": csi_total_size
    }

In [3]:
import struct



class OpenCVFrameData:

    def __init__(self, raw_data: bytes):
        self.frame_size = None  # long
        self.raw_size = None    # long
        self.timestamp = None   # long
        self.width = None       # int
        self.height = None      # int
        self.channels = None    # int
        self.reserved = None    # int
        self.raw = None         # bytes

        self.endian_format = '<'
    
        if raw_data is not None:
            self.parse(raw_data)

    def parse(self, raw_data: bytes):
        # Get the size of the data frame
        self.frame_size = struct.unpack(self.endian_format + 'Q', raw_data[0:8])[0]

        # Get the size of the raw data
        self.raw_size = struct.unpack(self.endian_format + 'Q', raw_data[8:16])[0]

        # Get the timestamp
        self.timestamp = struct.unpack(self.endian_format + 'Q', raw_data[16:24])[0]

        # Get the width
        self.width = struct.unpack(self.endian_format + 'i', raw_data[24:28])[0]

        # Get the height
        self.height = struct.unpack(self.endian_format + 'i', raw_data[28:32])[0]

        # Get the number of channels
        self.channels = struct.unpack(self.endian_format + 'i', raw_data[32:36])[0]

        # Get the reserved int
        self.reserved = struct.unpack(self.endian_format + 'i', raw_data[36:40])[0]

        
        print(self.frame_size, 
              self.raw_size, self.timestamp, 
              self.width, self.height, self.channels, 
              self.reserved)
#         # Get the raw data
#         if self.reserved == raw_data[40]:
#             self.raw = raw_data[40:]
#         else:
#             raise ValueError("Error: The reserved int is not equal to the raw data")

In [4]:
def parse_image_data(cv_data:bytes, cv_data_len:int):

    # Use a list to store the CV data frames
    cv_data_frames = []

    # Parse the CV data
    anchor = 0
    while cv_data_len > 0:

        # Get the size of the data frame
        data_frame_size = struct.unpack('<Q', cv_data[anchor:anchor + 8])[0]

        # Get the data frame
        data_frame = cv_data[anchor: anchor + data_frame_size]

        # Parse the data frame
        cv_data_frame = OpenCVFrameData(data_frame)

        # print(cv_data_frame.width, cv_data_frame.height, cv_data_frame.channels,
        #       cv_data_frame.frame_size, cv_data_frame.raw_size, len(cv_data_frame.raw))

        # Add the data frame to the list
        cv_data_frames.append(cv_data_frame)

        # Update the anchor and the data length
        anchor += data_frame_size
        cv_data_len -= data_frame_size

    return cv_data_frames

In [6]:
csv_info = parse_bin_file("./202308270743.bin")
cv_data_frames = parse_image_data(csv_info["cv_data"], csv_info["cv_data_len"])

57640 57600 1693147350113 320 180 1 0
57640 57600 1693147350183 320 180 1 0
57640 57600 1693147350227 320 180 1 0
57640 57600 1693147350266 320 180 1 0
57640 57600 1693147350307 320 180 1 0
57640 57600 1693147350352 320 180 1 0
57640 57600 1693147350392 320 180 1 0
57640 57600 1693147350430 320 180 1 0
57640 57600 1693147350468 320 180 1 0
57640 57600 1693147350506 320 180 1 0
57640 57600 1693147350543 320 180 1 0
57640 57600 1693147350581 320 180 1 0
57640 57600 1693147350619 320 180 1 0
57640 57600 1693147350657 320 180 1 0
57640 57600 1693147350694 320 180 1 0
57640 57600 1693147350733 320 180 1 0
57640 57600 1693147350770 320 180 1 0
57640 57600 1693147350811 320 180 1 0
57640 57600 1693147350850 320 180 1 0
57640 57600 1693147350888 320 180 1 0
57640 57600 1693147350928 320 180 1 0
57640 57600 1693147350966 320 180 1 0
57640 57600 1693147351006 320 180 1 0
57640 57600 1693147351045 320 180 1 0
57640 57600 1693147351082 320 180 1 0
57640 57600 1693147351120 320 180 1 0
57640 57600 

In [12]:
# show first frame in hex
fragment = csv_info["cv_data"][:100]

# frame_size raw_size, timestamp, width, height, channels, reserved
# print the above info in hex
print("frame_size raw_size, timestamp, width, height, channels, reserved")
print(fragment[:8].hex(), fragment[8:16].hex(), fragment[16:24].hex(), fragment[24:28].hex(), fragment[28:32].hex(), fragment[32:36].hex(), fragment[36:40].hex())

# print out the raw data and insert a new line every 16 bytes
for i in range(40, 100, 16):
    print(fragment[i:i+16].hex())

    

frame_size raw_size, timestamp, width, height, channels, reserved
28e1000000000000 00e1000000000000 614472378a010000 40010000 b4000000 01000000 00000000
29343c2530383441494c5c6352646b4f
656b50666c4e666c4c666c4c666c4a67
6c4b686d4e686e4e686e4e686e4f696f
4f69704f69704e6a714e6a71
